In [1]:
#!csharp

#r "nuget: xunit, 2.8.1"
using System.Threading;
using Xunit;

class Message
{
    static int messageHandled = 0;
    int id;
    public Message()
    {
        id = Thread.CurrentThread.ManagedThreadId;
    }

    public void Handle()
    {
        Interlocked.Increment(ref messageHandled);
        Assert.NotEqual(id, Thread.CurrentThread.ManagedThreadId);
    }

    public static void Check()
    {
        Assert.Equal(200, messageHandled);
    }
}

Installed Packages xunit, 2.8.1

In [3]:
#!csharp

using System.Collections.Concurrent;

var queue1 = new BlockingCollection<Message>(2); //очередь первого потока
var queue2 = new BlockingCollection<Message>(2); //очередь второго потока

static AutoResetEvent thrsig = new AutoResetEvent(false);
static AutoResetEvent thr1sig = new AutoResetEvent(true);

queue1.Add(new Message());

List<int> count = new List<int>();

Thread thr = new (() =>
{
    for(int i =1;i<101;i++)
    {
        thr1sig.WaitOne();
        var v = queue1.Take();
        v.Handle();
        queue2.Add(new Message());
        if(i==100) count.Add(i);
        thrsig.Set();
    }
    queue1.Take();
});

Thread thr1 = new (() =>
{
    for(int i =1;i<101;i++)
    {
        thrsig.WaitOne();
        var v = queue2.Take();
        v.Handle();
        queue1.Add(new Message());
        if(i==100) count.Add(i);
        thr1sig.Set();
    }
});

thr.Start();
thr1.Start();

thr.Join();
thr1.Join();

int i=1;
foreach(var v in count)
{
    Console.WriteLine($"Количество обработанных сообщений в {i} потоке = "+v);
    i++;
}

#!csharp

Assert.Equal(0, queue1.Count);
Assert.Equal(0, queue2.Count);
Message.Check();

Количество обработанных сообщений в 1 потоке = 100
Количество обработанных сообщений в 2 потоке = 100


Error: Xunit.Sdk.EqualException: Assert.Equal() Failure: Values differ
Expected: 200
Actual:   400
   at Xunit.Assert.Equal[T](T expected, T actual, IEqualityComparer`1 comparer) in /_/src/xunit.assert/Asserts/EqualityAsserts.cs:line 148
   at Xunit.Assert.Equal[T](T expected, T actual) in /_/src/xunit.assert/Asserts/EqualityAsserts.cs:line 83
   at Submission#2.Message.Check()
   at Submission#6.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)